In [ ]:
d =  [
      [
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ],
        [
          0.0,
          0.0
        ]
      ]
    ]

In [3]:
import os
import cv2
import json
import torch
from tqdm import tqdm
from ultralytics import YOLO

# === config ===
input_folder = '/home/siamai/data/chuniji/Tryweek7swin/videossliced_tom/videoslicedfull_aj'
output_folder = '/home/siamai/data/Penguin/week7/ajart/json_tom'
os.makedirs(output_folder, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = YOLO('yolo11n-pose.pt')
model.overrides['verbose'] = False  # ปิด verbose logs

# default keypoints (18 points, (0,0))
d = [[[0, 0] for _ in range(17)]]  # 1 person, 18 joints, x=0, y=0

# === loop over video files ===
for video_file in os.listdir(input_folder):
    if not video_file.endswith(('.mp4', '.avi', '.mov')):
        continue

    video_path = os.path.join(input_folder, video_file)
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    all_poses = []

    pbar = tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc=f'Processing {video_file}', leave=False)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose=False, device=0)  # ใช้ GPU 0
        if results[0].keypoints is not None and len(results[0].keypoints.xy) > 0:
            keypoints = results[0].keypoints.xy[0].cpu().numpy().tolist()  # คนแรก
            frame_data = {
                'frame_id': frame_id,
                'keypoints': [keypoints]
            }
        else:
            frame_data = {
                'frame_id': frame_id,
                'keypoints': d
            }

        all_poses.append(frame_data)
        frame_id += 1
        pbar.update(1)
    pbar.close()

    output_path = os.path.join(output_folder, video_file.rsplit('.', 1)[0] + '.json')
    with open(output_path, 'w') as f:
        json.dump(all_poses, f, indent=2)

    cap.release()


Using device: cuda


Processing s06_smiley_t3056.mp4:   0%|          | 0/150 [00:00<?, ?it/s]

In [4]:
fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Framerate of {video_file}: {fps} FPS")

Framerate of s02_smiley_t2028.mp4: 0.0 FPS


In [ ]:
import json
from ultralytics import YOLO
import cv2

# Load your JSON file containing image paths or frame info
with open('/home/siamai/data/Penguin/week7/output_json/s01_smiley_t0448.json', 'r') as f:
    data = json.load(f)

model = YOLO('yolov8n-pose.pt')

for item in data:
    # Assuming item['image_path'] contains the path to an image/frame
    img_path = item['image_path']
    frame = cv2.imread(img_path)

    results = model(frame)

    # process results
    keypoints = results[0].keypoints.xy if results[0].keypoints else []

    print(f"Keypoints for {img_path}: {keypoints.tolist()}")


In [5]:
import cv2
import json
import os

input_json_path = '/home/siamai/data/Penguin/week7/output_json/s01_smiley_t0448.json'
video_path = '/home/siamai/data/chuniji/Tryweek7swin/videossliced/s01_smiley_t0448.mp4'
output_video_path = 'output_with_keypoints.mp4'

# Load JSON keypoints
with open(input_json_path, 'r') as f:
    keypoints_data = json.load(f)

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

frame_idx = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    if frame_idx < len(keypoints_data):
        points = keypoints_data[frame_idx]['keypoints']  # List of lists of [x, y]
        
        # Draw keypoints
        for person_kp in points:
            for (x, y) in person_kp:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 0), -1)
    
    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print(f"Output saved to {output_video_path}")


Output saved to output_with_keypoints.mp4


In [ ]:
import cv2

video_path = 'path/to/your_video.mp4'
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Framerate: {fps} FPS")

cap.release()